# RMB Fractional Cover Zonal Stats Pipeline.

Copyright 2021 Robert McGregor

### cd to the directory from the Anaconda/Miniconda terminal.

cd E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline

Depending on your system you may need to add "/d"
i.e. cd /d E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline

This notebook controls the processing of fractonal cover zonal stats and produces the following outputs:
- static time series plot
- interactive time series plot
- fractonal cover zonal stats csv
- rainfall zonal stats csv

## Setup

## Required packages:
The ODK Aggregate processing pipeline requires the following package:

- os
- csv
- warinings
- geopandas
- datetime
- os
- argparse
- sys
- shutill
- sys
- glob
- pandas
- rasterstats
- matplotlib
- bokeh

If you are using this notebook locally, you may need to install these packages using conda or pip. Uncomment one of the cells below by removing the hashtag('#') and run the cell by pressing **Shift+Enter**.

## Lets check the full list of packages in your environment

In [ ]:
! conda list

## Lets check the core packages versions in your conda environment.

In [ ]:
import pandas; print(f"pandas: {pandas.__version__}")
import numpy; print(f"numpy: {numpy.__version__}")
import geopandas; print(f"geopandas: {geopandas.__version__}")
import argparse; print(f"argpass: {argparse.__version__}")
import selenium; print(f"selenium: {selenium.__version__}")
import rasterstats; print(f"rasterstats: {rasterstats.__version__}")
import matplotlib; print(f"matplotlib: {matplotlib.__version__}")
import bokeh; print(f"bokeh: {bokeh.__version__}")


## Don't have a package?

In [ ]:
# If you do not have the required packages uncomment out the required line if you are on you local computer you will 
# need use external wifi (i.e. hotspot your phone)

#! conda install -c conda-forge numpy geopandas argparse rasterstats matplotlib bokeh
#! conda install -c conda-forge numpy
#! conda install -c conda-forge geopandas
#! conda install -c conda-forge argparse
#! conda install -c conda-forge rasterstats
#! conda install -c conda-forge matplotlib
#! conda install -c conda-forge bokeh

## Command arguments


- '-t', '--tile_grid', help='Enter filepath for the Landsat Tile Grid.shp.', default is set to r"E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline\assets\shapefiles\Landsat_wrs2_TileGrid.shp"
- '-d', '--directory_odk', help='The directory containing the ODK output excel files.'
- '-x', '--export_dir', help='Enter the export directory for all of the final outputs.', default is set to, r"Z:\DENR_Satellite_Imagery\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs"
- '-i', '--image_count', type=int, help='Enter the minimum amount of Landsat images required per tile as an integer (i.e. 800).', default is set to 800
- '-l', '--landsat_dir', help='Enter the Landsat Directory path', default is set to 'Z:\Landsat\wrs2'.
- '-n', '--no_data', help='Enter the Landsat Fractional Cover no data value (i.e. 0)', default is set to 0.
- '-r', '--rainfall_dir', help='Enter the rainfall image directory path.', default is set to r'E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline\assets\rainfall_raster'.
- '-s1', '--search_criteria1', help='Enter the end name search criteria string for the fractional cover Landsat images (i.e. 'dilm2_zstdmask.img').', default is set to 'dilm2_zstdmask.img'.
- '-s2', '--search_criteria2', help='Enter the end name search criteria string for the fractional cover Landsat images (i.e. 'dilm3_zstdmask.img').', default is set to 'dilm3_zstdmask.img'.
- '-s3', '--search_criteria3', help='Enter the end name search criteria string for the QLD Rainfall images (i.e. '.img').', default='.img'.
- '-e', '--end_date', help='Final date for the rainfall data override (i.e.2020-08-31) Do not enter if you want the script to determine the finish date..', default is set to '2020-08-31'.
- '-v', '--visits', help='Path to the latest integrated site shapefile containing previous visit information (default).', default is set to r'E:\DENR\code\rangeland_monitoring\rmb_zonal_plots_ploting_pipeline\assets\shapefiles\NT_StarTransect_20200713.shp'.
- '-p', '--pastoral_estate', help='File path to the pastoral estate shapefile.', default is set to  r'E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline\assets\shapefiles\pastoral_estate.shp'.
- '-m', '--rolling_mean', help='Integer value (i.e 3 or 5) to create the rolling mean of the date.', default is set to 5.

•	Tile grid - The file path to the Landsat Tile Grid shapefile.
    Default: E:\DENR\code\rangeland_monitoring\rmb_fractional_cover_zonal_stats_pipeline\assets\shapefiles\Landsat_wrs2_TileGrid.shp

•	Directory odk - The directory containing the ODK output excel files at a property level, or the complete output derived from RMB Aggregate Processing Pipeline.
    No default

•	Export dir - The export directory for all of outputs (temporary folder).
    Default:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs

•	Image count - The minimum amount of Landsat images required per tile as an integer (i.e. 950).
     Default: 950 (default is updated annually) 
     
•	Landsat dir – The file path to the Landsat tile directory
    Default: Z:\Landsat\wrs2
    
•	No data - The Landsat Fractional Cover no data value (i.e. 0)
    Default: 0
    
•	Rainfall dir - The file path to the rainfall raster image directory.
    Default: Z:\Scratch\Zonal_Stats_Pipeline\rainfall
•	Search criteria 1 - The end name search criteria string for Landsat Fractional Cover rasters (i.e. dilm2_zstdmask.img).
    Default: dilm2_zstdmask.img
    
•	Search criteria 2 - The end name search criteria string for Landsat Fractional Cover rasters (i.e. dilm3_zstdmask.img).
    Default: dilm3_zstdmask.img
    
•	Search criteria 3 - The end name search criteria string for QLD Gov derived rainfall images (i.e. '.img').
    Default: '.img'
    
•	End date - Final date for the plotting of rainfall data.
    Default: 2021-08-31
    
•	Visits – File path to the latest integrated site shapefile containing previous visit history.
    Default: E:\DENR\code\rangeland_monitoring\rmb_zonal_plots_ploting_pipeline\assets\shapefiles\NT_StarTransect_20200713.shp
    
•	Pastoral estate - File path to the pastoral estate shapefile.
    Default: E:\DENR\code\rangeland_monitoring\rmb_fractional_cover_zonal_stats_pipeline\assets\shapefiles\pastoral_estate.shp
    
•	Rolling mean - Integer value (i.e 3 or 5) to create the plotting rolling mean of the date.
    Default: 5
    
•	Pastoral districts dir - File path to the Pastoral Districts directory.
    Default: Z:\Scratch\Zonal_Stats_Pipeline\rmb_aggregate_processing\Pastoral_Districts


#### Known issues

- Running this script outside of the remote desktop will be slow especially for the regions. 
    - Alice Springs should only run on the remote server.
    - Katherine is unknown at this time.
    - Darwin can run it locally or on the remote server.
    
- Running this script on the remote server in the base Anaconda environment will not connect to the reproject database; as such, you will have false projection shapefiles.
- Jupyter notebooks holds items in memory, if you change a command argument, you may need to end the Kernel (Restart & Clear Output).


### Check that you have connected to the project database

In [1]:
import geopandas as gpd
import os

path_parent = os.path.dirname(os.getcwd())
previous_visists_shapefile =  r"C:\Users\uavra\image_processing\shapefiles\NT_Pastoral_Estate.shp"
gdf = gpd.read_file(previous_visists_shapefile)
gdf.crs

<Geographic 2D CRS: GEOGCS["GDA94",DATUM["D_GDA_1994",SPHEROID["GRS_19 ...>
Name: GDA94
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: Geocentric Datum of Australia 1994
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

##### Do your results look like this?

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich


If yes, you are ready to run some code.

If not, open this notebook from within the conda environment zonal, and try again.

Still having issues? Contact Rob or Grant on Teams, sharing your screen to trouble shoot.

## Run the RMB Fractional Cover Zonal Stats Pipeline

#### Change your working directory

In [1]:
import os
path_parent = os.path.dirname(os.getcwd())
code_dir =  path_parent +  '\\code'
os.chdir(code_dir)
print(os.getcwd())
# path should be: E:\DENR\code\rangeland_monitoring\fractional_cover_zonal_stats_pipeline\code

E:\DENR\code\rangeland_monitoring\rmb_fractional_cover_zonal_stats_pipeline_tif\code


In [2]:
#run step1_1_initiate_fractional_cover_zonal_stats_pipeline.py -h

In [3]:
run step1_1_initiate_fractional_cover_zonal_stats_pipeline.py -d U:\Pastoral_Districts\VRD\WLM_Wallamunga\Data\Processed_Odk\Property\2021\Csv

The following temporary directory will be created:  \rmcgr_20211019_0751
The following export directory will be created:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751
--------------------------------------------------
rain_finish_date:  2021-09-30
The following ODK forms were identified for processing:
 - Integrated:  1
tile_status_dir: Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\tile_status
Confirm that there are sufficient fractional cover tiles for processing
landsat_tile_dir:  Z:\Landsat\wrs2\106_073
 - Total fractional cover tiles located:  991
 - Minimum tiles (command argument):  950
--------------------------------------------------
zonal_stats_output:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\zonal_stats
Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\tile_status\for_processing\106_073_

final_tile:  106073
complete_tile:  106073
start_date:  1988-05-01
finish_date:  2021-09-30
complete_tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
plot_bare_g complete tile:  106073
PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
file_path:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\zonal_stats\WLM_Wallamunga_106073_zonal_stats.csv
file:  WLM_Wallamunga_106073_zonal_stats.csv
--------------------------------------------------
tile_:  106073
ind_prop_output:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\prop_output\Wallamunga
line 72 file output:  Z:\Scratch\Zonal_Stats_Pipeline\rmb_fractional_cover_zonal_stats\outputs\rmcgr_20211019_0751\prop